## IMPORT DES LIBRAIRIES

pip install pandas
pip install sklearn
pip install matplotlib
pip install seaborn
pip install plotly
pip install scikit-plot
pip install xlrd

In [30]:
# Package de manipaulation des tableaux et dataframe
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Package pour analyse statistique
import scipy.stats as ss

# Package pour représentation graphique
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import scikitplot as skplt

# Affichage Image (png, jpeg, html ...)
from IPython.display import Image, HTML

# Typing des fonctions
from typing import List, Optional

from functools import reduce

Décision 1 : importer les fichiers 1 par 1 dans une DF car la mise en forme est différente pour chacun des fichiers

## DECLARATION DES FONCTIONS

In [24]:
# ---------------------------------------Fonction-------------------
# Création d'une dataframe de distribution 
def ditrib_calc(var1,var2,var3):
    df_temp = pd.DataFrame(var1.value_counts(normalize=True) * 100).reset_index().rename(columns={"index": var2, var3: "count"})
    return df_temp

# Représentation graphique : bar plot target

def ditrib_graph(var1,var2,var3):
    fig = px.bar(var1, 
                 y="target", x="count",
                 hover_data={'count':':.2f'}
    )
    fig.update_layout(
        title=var2,
        xaxis_title="Pourcentage par classe",
        yaxis_title=var3,
        margin=dict(l=0, r=0, t=30, b=50),
        width=500, height=300
    )
    return fig.show()

def del_top_lines(nb,df_init):
    # Fonction qui supprime : 
    # - les première lignes d'un tableau, 
    # - renomme les colonnes
    # - crée un nouvel index en préservant l'ancien
    #
    #
    # nb : index number of the new head table to keep
    # df_init : dataframe to change
    print(".......index new head number : ", nb)
    df_fin_trans = df_init.iloc[nb:]
    df_fin_trans.columns = df_fin_trans.values[0]
    df_fin_trans = df_fin_trans.iloc[1:].reset_index().rename(columns={"index": "old_index"})
    return df_fin_trans



# Dictionnaire des états
dict_states = {"United States" : "US", "Alabama" : "AL", "Alaska" : "AK", "Arizona" : "AZ", "Arkansas" : "AR", "California" : "CA", "Colorado" : "CO", "Connecticut" : "CT", "Delaware" : "DE", "District of Columbia" : "DC", "Florida" : "FL", "Georgia" : "GA", "Hawaii" : "HI", "Idaho" : "ID", "Illinois" : "IL", "Indiana" : "IN", "Iowa" : "IA", "Kansas" : "KS", "Kentucky" : "KY", "Louisiana" : "LA", "Maine" : "ME", "Maryland" : "MD", "Massachusetts" : "MA", "Michigan" : "MI", "Minnesota" : "MN", "Mississippi" : "MS", "Missouri" : "MO", "Montana" : "MT", "Nebraska" : "NE", "Nevada" : "NV", "New Hampshire" : "NH", "New Jersey" : "NJ", "New Mexico" : "NM", "New York" : "NY", "North Carolina" : "NC", "North Dakota" : "ND", "Ohio" : "OH", "Oklahoma" : "OK", "Oregon" : "OR", "Pennsylvania" : "PA", "Rhode Island" : "RI", "South Carolina" : "SC", "South Dakota" : "SD", "Tennessee" : "TN", "Texas" : "TX", "Utah" : "UT", "Vermont" : "VT", "Virginia" : "VA", "Washington" : "WA", "West Virginia" : "WV", "Wisconsin" : "WI", "Wyoming" : "WY"}

Décision 2 : le pilier de cette étude est l'index unique de chaque était ie le code FIPS. Notre fichier principal de résultat electoral ne contient que les informations des états ayant un code FIPS entre 1 et 56.
Décision 3 : les colonnes à gader :
---- Education : 
--------- Rural et influence code 2013
--------- Données entre 2015 et 2019
---- Pop est : 
--------- Rural et influence code 2013
--------- Données entre 2015 et 2019
---- Pov est : 
--------- Rural et influence code 2013
--------- Estimation (et non intervalle de confiance sup et inf)



## IMPORT et MISE EN FORME DES DONNEES

In [48]:
#---------------------------------------IMPORT DES DONNEES----------------------------------------------------------
# On spécifie le chemin d'accès aux données
folder_path = "./data/"

# On importe le fichier principal
df_pres_result = pd.read_csv(folder_path + "2020_US_County_Level_Presidential_Results.csv", sep = ",")
df_pres_result_old = pd.read_csv(folder_path + "US_County_Level_Presidential_Results_08-16.csv", sep = ",")
df_education = pd.read_excel(folder_path + "Education.xls")
df_pop_est = pd.read_excel(folder_path + "PopulationEstimates.xls")
df_pov_est = pd.read_excel(folder_path + "PovertyEstimates.xls")
df_unemp = pd.read_excel(folder_path + "Unemployment.xls")

#-----------------------------------------TYPOLOGIE DES FEATURES--------------------------------------------------------


# Mise en forme et Modification des entêtes des fichiers excels 
print(".... Mise en forme et Modification des entêtes des fichiers excels")
df_education = del_top_lines(3,df_education)
df_pop_est = del_top_lines(1,df_pop_est)
df_pov_est = del_top_lines(3,df_pov_est)
df_unemp = del_top_lines(3,df_unemp)

# Uniformisation du code FIPS du fichier CSV
print(".... Uniformisation du code FIPS")
df_pres_result['county_fips'] = df_pres_result['county_fips'].astype(str)
df_pres_result['county_fips'] = df_pres_result['county_fips'].apply(lambda x: '0' + x if len(x) == 4 else x)
#print(df_pres_result['county_fips'])

update_name_col_1 = 'FIPS_code'
update_name_col_2 = 'State_code'
update_name_col_3 = 'Area_name'
update_name_col_4 = 'Rural_urban_continuum_code_2013'
update_name_col_5 = 'Urban_influence_code_2013'
update_name_col_6 = 'State_name'

# Uniformisation des noms de colonnes communes à chaque fichier
print(".... Uniformisation des noms de colonnes communes à chaque fichier")
df_pres_result.rename(columns={'county_fips': update_name_col_1, 'state_name': update_name_col_6, 'county_name' : update_name_col_3}, inplace=True)
df_education.rename(columns={'FIPS Code': update_name_col_1, 'State': update_name_col_2, 'Area name' : update_name_col_3, '2013 Rural-urban Continuum Code' : update_name_col_4, '2013 Urban Influence Code' : update_name_col_5}, inplace=True)
df_pop_est.rename(columns={'FIPStxt': update_name_col_1, 'State': update_name_col_2, 'Area_Name' : update_name_col_3, 'Rural-urban_Continuum Code_2013' : update_name_col_4, 'Urban_Influence_Code_2013' : update_name_col_5}, inplace=True)
df_pov_est.rename(columns={'FIPStxt': update_name_col_1, 'Stabr': update_name_col_2, 'Area_name' : update_name_col_3, 'Rural-urban_Continuum_Code_2013' : update_name_col_4, 'Urban_Influence_Code_2013' : update_name_col_5}, inplace=True)
df_unemp.rename(columns={'fips_txt': update_name_col_1, 'Stabr': update_name_col_2, 'area_name' : update_name_col_3, 'Rural_urban_continuum_code_2013' : update_name_col_4, 'Urban_influence_code_2013' : update_name_col_5}, inplace=True)

# Création d'une nouvelle colonne 
print(".... Création d'une nouvelle colonne : ", update_name_col_2)
df_pres_result[update_name_col_2] = df_pres_result[update_name_col_6]
df_pres_result[update_name_col_2]=df_pres_result[update_name_col_2].replace(dict_states)

# Ajout de nouvelles lignes (niveau ETATS) dans la DF des résultats de 2020 afin de garder une cohérence avec les autrs fichiers
print(".... Ajout du niveau 'état' dans le fichier cible")
df_states = pd.DataFrame({update_name_col_1 : ["00000", "01000", "02000", "04000", "05000", "06000", "08000", "09000", "10000", "11000", "12000", "13000", "15000", "16000", "17000", "18000", "19000", "20000", "21000", "22000", "23000", "24000", "25000", "26000", "27000", "28000", "29000", "30000", "31000", "32000", "33000", "34000", "35000", "36000", "37000", "38000", "39000", "40000", "41000", "42000", "44000", "45000", "46000", "47000", "48000", "49000", "50000", "51000", "53000", "54000", "55000", "56000"],
                          update_name_col_2 : ["US", "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"],
                          update_name_col_3 : ["United States", "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Lousiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"],
                          update_name_col_6 : ["United States", "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Lousiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]})
df_pres_result = pd.concat([df_pres_result, df_states])

# Selection du périmètre du premier périmètre d'analyse
# Attention : code FIPS, State, area, rural et urban influance déjà uniforme
print(".... Selection du périmètre du premier périmètre d'analyse")
col_educ_perim = [update_name_col_1,
                  #update_name_col_2,
                  #update_name_col_3,
                  update_name_col_4,
                  update_name_col_5,
                  "Less than a high school diploma, 2015-19",
                  "High school diploma only, 2015-19",
                  "Some college or associate's degree, 2015-19",
                  "Bachelor's degree or higher, 2015-19",
                  "Percent of adults with less than a high school diploma, 2015-19",
                  "Percent of adults with a high school diploma only, 2015-19",
                  "Percent of adults completing some college or associate's degree, 2015-19",
                  "Percent of adults with a bachelor's degree or higher, 2015-19"]

col_pop_est_perim = [update_name_col_1,
                     #update_name_col_2,
                     #update_name_col_3,
                     #update_name_col_4,
                     #update_name_col_5,
                     "Economic_typology_2015",
                     "POP_ESTIMATE_2015",
                     "POP_ESTIMATE_2016",
                     "POP_ESTIMATE_2017",
                     "POP_ESTIMATE_2018",
                     "POP_ESTIMATE_2019",
                     "N_POP_CHG_2015",
                     "N_POP_CHG_2016",
                     "N_POP_CHG_2017",
                     "N_POP_CHG_2018",
                     "N_POP_CHG_2019",
                     "Births_2015",
                     "Births_2016",
                     "Births_2017",
                     "Births_2018",
                     "Births_2019",
                     "Deaths_2015",
                     "Deaths_2016",
                     "Deaths_2017",
                     "Deaths_2018",
                     "Deaths_2019",
                     "NATURAL_INC_2015",
                     "NATURAL_INC_2016",
                     "NATURAL_INC_2017",
                     "NATURAL_INC_2018",
                     "NATURAL_INC_2019",
                     "INTERNATIONAL_MIG_2015",
                     "INTERNATIONAL_MIG_2016",
                     "INTERNATIONAL_MIG_2017",
                     "INTERNATIONAL_MIG_2018",
                     "INTERNATIONAL_MIG_2019",
                     "DOMESTIC_MIG_2015",
                     "DOMESTIC_MIG_2016",
                     "DOMESTIC_MIG_2017",
                     "DOMESTIC_MIG_2018",
                     "DOMESTIC_MIG_2019",
                     "NET_MIG_2015",
                     "NET_MIG_2016",
                     "NET_MIG_2017",
                     "NET_MIG_2018",
                     "NET_MIG_2019",
                     "RESIDUAL_2015",
                     "RESIDUAL_2016",
                     "RESIDUAL_2017",
                     "RESIDUAL_2018",
                     "RESIDUAL_2019",
                     "GQ_ESTIMATES_2015",
                     "GQ_ESTIMATES_2016",
                     "GQ_ESTIMATES_2017",
                     "GQ_ESTIMATES_2018",
                     "GQ_ESTIMATES_2019",
                     "R_birth_2015",
                     "R_birth_2016",
                     "R_birth_2017",
                     "R_birth_2018",
                     "R_birth_2019",
                     "R_death_2015",
                     "R_death_2016",
                     "R_death_2017",
                     "R_death_2018",
                     "R_death_2019",
                     "R_NATURAL_INC_2015",
                     "R_NATURAL_INC_2016",
                     "R_NATURAL_INC_2017",
                     "R_NATURAL_INC_2018",
                     "R_NATURAL_INC_2019",
                     "R_INTERNATIONAL_MIG_2015",
                     "R_INTERNATIONAL_MIG_2016",
                     "R_INTERNATIONAL_MIG_2017",
                     "R_INTERNATIONAL_MIG_2018",
                     "R_INTERNATIONAL_MIG_2019",
                     "R_DOMESTIC_MIG_2015",
                     "R_DOMESTIC_MIG_2016",
                     "R_DOMESTIC_MIG_2017",
                     "R_DOMESTIC_MIG_2018",
                     "R_DOMESTIC_MIG_2019",
                     "R_NET_MIG_2015",
                     "R_NET_MIG_2016",
                     "R_NET_MIG_2017",
                     "R_NET_MIG_2018",
                     "R_NET_MIG_2019"]

col_pov_est_perim = [update_name_col_1,
                     #update_name_col_2,
                     #update_name_col_3,
                     #update_name_col_4,
                     #update_name_col_5,
                     "POVALL_2019", "PCTPOVALL_2019", 
                     "POV017_2019", "PCTPOV017_2019", 
                     "POV517_2019", "PCTPOV517_2019", 
                     "MEDHHINC_2019", 
                     "POV04_2019", 
                     "PCTPOV04_2019"]

col_unemp_perim = [update_name_col_1,
                   #update_name_col_2,
                   #update_name_col_3,
                   #update_name_col_4,
                   #update_name_col_5,
                   "Metro_2013", 
                   "Civilian_labor_force_2015", 
                   "Employed_2015", 
                   "Unemployed_2015", 
                   "Unemployment_rate_2015", 
                   "Civilian_labor_force_2016", 
                   "Employed_2016", 
                   "Unemployed_2016", 
                   "Unemployment_rate_2016", 
                   "Civilian_labor_force_2017", 
                   "Employed_2017", 
                   "Unemployed_2017", 
                   "Unemployment_rate_2017", 
                   "Civilian_labor_force_2018", 
                   "Employed_2018", 
                   "Unemployed_2018", 
                   "Unemployment_rate_2018", 
                   "Employed_2019", 
                   "Unemployed_2019", 
                   "Unemployment_rate_2019", 
                   "Median_Household_Income_2019", 
                   "Med_HH_Income_Percent_of_State_Total_2019"] 

print(".... Réduction du scope : selection du périmètre du premier périmètre d'analyse des fichiers excels")
df_education = df_education[col_educ_perim]
df_pop_est = df_pop_est[col_pop_est_perim]
df_pov_est = df_pov_est[col_pov_est_perim]
df_unemp = df_unemp[col_unemp_perim]


# création d'une liste des dataframe à traiter
print(".... Création d'une liste des dataframe à traiter")
df_list = [df_pres_result,df_education,df_pop_est,df_pov_est,df_unemp]

# Info et affichage premières lignes 
#print(".... Info et affichage premières lignes")
#for d in df_list:
    #print(d.info())
    #display(d.head())

# Jointure en boucle sur toutes les DF sur l'élément update_name_col_1
print(".... Jointure en boucle sur toutes les DF sur l'élément : ", update_name_col_1)
df_merged = pd.DataFrame()
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=[update_name_col_1],how='inner'), df_list)

# Conversion de données
list_object = ['State_name', 'FIPS_code','Area_name','State_code', 'Rural_urban_continuum_code_2013','Urban_influence_code_2013']
print(".... Conversion de toutes les données au format numérique sauf : ", list_object)
list_all_col = list(df_merged.columns)
for col in list_all_col :
    if col not in list_object:
        df_merged[col] = df_merged[col].astype(float)

print(df_merged.info(verbose=True, show_counts=True))
display(df_merged.head())

.... Mise en forme et Modification des entêtes des fichiers excels
.......index new head number :  3
.......index new head number :  1
.......index new head number :  3
.......index new head number :  3
.... Uniformisation du code FIPS
.... Uniformisation des noms de colonnes communes à chaque fichier
.... Création d'une nouvelle colonne :  State_code
.... Ajout du niveau 'état' dans le fichier cible
.... Selection du périmètre du premier périmètre d'analyse
.... Réduction du scope : selection du périmètre du premier périmètre d'analyse des fichiers excels
.... Création d'une liste des dataframe à traiter
.... Jointure en boucle sur toutes les DF sur l'élément :  FIPS_code
.... Conversion de toutes les données au format numérique sauf :  ['State_name', 'FIPS_code', 'Area_name', 'State_code', 'Rural_urban_continuum_code_2013', 'Urban_influence_code_2013']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3164 entries, 0 to 3163
Data columns (total 133 columns):
 #    Column             

,State_name,FIPS_code,Area_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,...,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,Alabama,01001,Autauga County,19838.0,7503.0,27770.0,12335.0,0.714368,0.270184,0.444184,...,3.9,26196.0,25261.0,935.0,3.6,25458.0,714.0,2.7,58233.0,112.481888
1,Alabama,01003,Baldwin County,83544.0,24578.0,109679.0,58966.0,0.761714,0.224090,0.537623,...,4.1,95233.0,91809.0,3424.0,3.6,94675.0,2653.0,2.7,59871.0,115.645828
2,Alabama,01005,Barbour County,5622.0,4816.0,10518.0,806.0,0.534512,0.457882,0.076631,...,5.8,8414.0,7987.0,427.0,5.1,8213.0,324.0,3.8,35972.0,69.482918
3,Alabama,01007,Bibb County,7525.0,1986.0,9595.0,5539.0,0.784263,0.206983,0.577280,...,4.4,8605.0,8268.0,337.0,3.9,8419.0,266.0,3.1,47918.0,92.557610
4,Alabama,01009,Blount County,24711.0,2640.0,27588.0,22071.0,0.895716,0.095694,0.800022,...,4.0,25069.0,24201.0,868.0,3.5,24655.0,676.0,2.7,52902.0,102.184624


In [ ]:
#----------------------------------------------CREATION DE LA TARGET 'Party' ---------------------------------------------------
conditionlist = [
    (df_pres_result['per_gop'] < df_pres_result['per_dem']) ,
    (df_pres_result['per_gop'] > df_pres_result['per_dem'])]
choicelist = [0,1]
df_pres_result['party'] = np.select(conditionlist, choicelist, default='Not Specified')

#print(df_pres_result['party']) #Affichage de la colonne party

# -------------------------------Representation globale de la target party-------------------

target_distribution_bef_split = ditrib_calc(df_pres_result['party'],"target","party")
#print(target_distribution_bef_split)
ditrib_graph(target_distribution_bef_split,"Repartition de la target global","Target (GOP/DEM)")

In [ ]:
#-------------------------------------------- REGROUPEMENT DES DONNES SOCIOECONOMIQUES -----------------------

#-------------------------------------------- MISE EN FORME DU JEU DE DONNEES -----------------------
# Séparer les variables explicatives de la target
X = df_pres_result.drop(["party"], axis=1)
y = df_pres_result["party"]

# split des données en train et test
set_seed = 1204
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=set_seed)

# Création des listes de variables par catégorie
cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()
num_cols =  X_train.select_dtypes(include=np.number).columns.tolist()
all_cols = X_train.columns.tolist()


# ------------------------------------DISTRIBUTION DE LA feature target 'party'-----------------------------------------------
target_distribution = ditrib_calc(y_train,"target","party")
#print(target_distribution)
ditrib_graph(target_distribution,"Repartition de la target sur le jeu y train","Target (GOP/DEM)")

In [21]:
df_pres_result.to_csv('./export_df.csv', index = False, header=True)

In [ ]:
df_pres_result=df_pres_result.append({update_name_col_2 : "US" , update_name_col_1 : "00000", update_name_col_3 : "United States", update_name_col_6 : "United States"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "AL" , update_name_col_1 : "01000", update_name_col_3 : "Alabama", update_name_col_6 : "Alabama"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "AK" , update_name_col_1 : "02000", update_name_col_3 : "Alaska", update_name_col_6 : "Alaska"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "AZ" , update_name_col_1 : "04000", update_name_col_3 : "Arizona", update_name_col_6 : "Arizona"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "AR" , update_name_col_1 : "05000", update_name_col_3 : "Arkansas", update_name_col_6 : "Arkansas"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "CA" , update_name_col_1 : "06000", update_name_col_3 : "California", update_name_col_6 : "California"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "CO" , update_name_col_1 : "08000", update_name_col_3 : "Colorado", update_name_col_6 : "Colorado"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "CT" , update_name_col_1 : "09000", update_name_col_3 : "Connecticut", update_name_col_6 : "Connecticut"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "DE" , update_name_col_1 : "10000", update_name_col_3 : "Delaware", update_name_col_6 : "Delaware"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "DC" , update_name_col_1 : "11000", update_name_col_3 : "District of Columbia", update_name_col_6 : "District of Columbia"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "FL" , update_name_col_1 : "12000", update_name_col_3 : "Florida", update_name_col_6 : "Florida"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "GA" , update_name_col_1 : "13000", update_name_col_3 : "Georgia", update_name_col_6 : "Georgia"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "HI" , update_name_col_1 : "15000", update_name_col_3 : "Hawaii", update_name_col_6 : "Hawaii"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "ID" , update_name_col_1 : "16000", update_name_col_3 : "Idaho", update_name_col_6 : "Idaho"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "IL" , update_name_col_1 : "17000", update_name_col_3 : "Illinois", update_name_col_6 : "Illinois"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "IN" , update_name_col_1 : "18000", update_name_col_3 : "Indiana", update_name_col_6 : "Indiana"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "IA" , update_name_col_1 : "19000", update_name_col_3 : "Iowa", update_name_col_6 : "Iowa"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "KS" , update_name_col_1 : "20000", update_name_col_3 : "Kansas", update_name_col_6 : "Kansas"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "KY" , update_name_col_1 : "21000", update_name_col_3 : "Kentucky", update_name_col_6 : "Kentucky"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "LA" , update_name_col_1 : "22000", update_name_col_3 : "Louisiana", update_name_col_6 : "Louisiana"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "ME" , update_name_col_1 : "23000", update_name_col_3 : "Maine", update_name_col_6 : "Maine"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "MD" , update_name_col_1 : "24000", update_name_col_3 : "Maryland", update_name_col_6 : "Maryland"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "MA" , update_name_col_1 : "25000", update_name_col_3 : "Massachusetts", update_name_col_6 : "Massachusetts"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "MI" , update_name_col_1 : "26000", update_name_col_3 : "Michigan", update_name_col_6 : "Michigan"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "MN" , update_name_col_1 : "27000", update_name_col_3 : "Minnesota", update_name_col_6 : "Minnesota"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "MS" , update_name_col_1 : "28000", update_name_col_3 : "Mississippi", update_name_col_6 : "Mississippi"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "MO" , update_name_col_1 : "29000", update_name_col_3 : "Missouri", update_name_col_6 : "Missouri"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "MT" , update_name_col_1 : "30000", update_name_col_3 : "Montana", update_name_col_6 : "Montana"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "NE" , update_name_col_1 : "31000", update_name_col_3 : "Nebraska", update_name_col_6 : "Nebraska"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "NV" , update_name_col_1 : "32000", update_name_col_3 : "Nevada", update_name_col_6 : "Nevada"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "NH" , update_name_col_1 : "33000", update_name_col_3 : "New Hampshire", update_name_col_6 : "New Hampshire"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "NJ" , update_name_col_1 : "34000", update_name_col_3 : "New Jersey", update_name_col_6 : "New Jersey"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "NM" , update_name_col_1 : "35000", update_name_col_3 : "New Mexico", update_name_col_6 : "New Mexico"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "NY" , update_name_col_1 : "36000", update_name_col_3 : "New York", update_name_col_6 : "New York"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "NC" , update_name_col_1 : "37000", update_name_col_3 : "North Carolina", update_name_col_6 : "North Carolina"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "ND" , update_name_col_1 : "38000", update_name_col_3 : "North Dakota", update_name_col_6 : "North Dakota"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "OH" , update_name_col_1 : "39000", update_name_col_3 : "Ohio", update_name_col_6 : "Ohio"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "OK" , update_name_col_1 : "40000", update_name_col_3 : "Oklahoma", update_name_col_6 : "Oklahoma"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "OR" , update_name_col_1 : "41000", update_name_col_3 : "Oregon", update_name_col_6 : "Oregon"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "PA" , update_name_col_1 : "42000", update_name_col_3 : "Pennsylvania", update_name_col_6 : "Pennsylvania"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "RI" , update_name_col_1 : "44000", update_name_col_3 : "Rhode Island", update_name_col_6 : "Rhode Island"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "SC" , update_name_col_1 : "45000", update_name_col_3 : "South Carolina", update_name_col_6 : "South Carolina"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "SD" , update_name_col_1 : "46000", update_name_col_3 : "South Dakota", update_name_col_6 : "South Dakota"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "TN" , update_name_col_1 : "47000", update_name_col_3 : "Tennessee", update_name_col_6 : "Tennessee"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "TX" , update_name_col_1 : "48000", update_name_col_3 : "Texas", update_name_col_6 : "Texas"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "UT" , update_name_col_1 : "49000", update_name_col_3 : "Utah", update_name_col_6 : "Utah"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "VT" , update_name_col_1 : "50000", update_name_col_3 : "Vermont", update_name_col_6 : "Vermont"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "VA" , update_name_col_1 : "51000", update_name_col_3 : "Virginia", update_name_col_6 : "Virginia"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "WA" , update_name_col_1 : "53000", update_name_col_3 : "Washington", update_name_col_6 : "Washington"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "WV" , update_name_col_1 : "54000", update_name_col_3 : "West Virginia", update_name_col_6 : "West Virginia"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "WI" , update_name_col_1 : "55000", update_name_col_3 : "Wisconsin", update_name_col_6 : "Wisconsin"} , ignore_index=True)
df_pres_result=df_pres_result.append({update_name_col_2 : "WY" , update_name_col_1 : "56000", update_name_col_3 : "Wyoming", update_name_col_6 : "Wyoming"} , ignore_index=True)